In [1]:
!pip install scikit-learn


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report


In [3]:
image_dir = Path(r'/Users/dipak/Downloads/archive (10)/images')  #Points to your food images folder
filepaths = list(image_dir.glob(r'**/*.jpg'))  # finds all .jpg files recursively in all subfolders Result: A list of all image paths
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
'''For each image path, it extracts the parent folder name as the label (food type)
Example: If image is at food/images/pizza/image1.jpg, the label is "pizza"
This assumes each food type is in its own subfolder'''
filepaths = pd.Series(filepaths, name='Filepath').astype(str) #Combines filepaths and labels into a pandas DataFrame with two columns: Filepath and Label
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
category_samples = [] #For each unique food category, it randomly selects 100 images
for category in images['Label'].unique(): 
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
image_df = pd.concat(category_samples, axis=0).sample(
    frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
image_df['Label'].value_counts()

Label
pork_chop        100
bread_pudding    100
club_sandwich    100
french_fries     100
beef_tartare     100
                ... 
creme_brulee     100
hummus           100
clam_chowder     100
caprese_salad    100
foie_gras        100
Name: count, Length: 101, dtype: int64

In [5]:
train_df, test_df = train_test_split(
image_df, train_size=0.7, shuffle=True, random_state=42)
#Splits your balanced dataset into training (70%) and testing (30%) sets
#shuffle=True randomly mixes before splitting (prevents sequential bias)

In [ ]:
# Enhanced data augmentation for better generalization
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    # Data augmentation to artificially expand the dataset
    rotation_range=30,  # Randomly rotate images by up to 30 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Apply random shearing
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in new pixels after transformations
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [7]:
'''This code is the bridge between your data and the model. It loads actual image files from disk, preprocesses them, and organizes them into batches ready for training. Let me explain each part:

What flow_from_dataframe() Does
It reads image files from your DataFrame and converts them into batches that TensorFlow can use. Think of it as a smart data loader.'''

train_images = train_generator.flow_from_dataframe(
dataframe=train_df,
x_col='Filepath',
y_col='Label',
target_size=(224, 224),
color_mode='rgb',
class_mode='categorical',
batch_size=32,
shuffle=True,
seed=42,
subset='training'
)
'''Loads images from train_df paths

Resizes each to 224×224 (MobileNetV2's expected input size)

Shuffles order to prevent overfitting to image sequence

Creates batches of 32 images at a time

Subset='training' means: Use only the 80% portion (excluding validation)
'''
val_images = train_generator.flow_from_dataframe(
dataframe=train_df,
x_col='Filepath',
y_col='Label',
target_size=(224, 224),
color_mode='rgb',
class_mode='categorical',
batch_size=32,
shuffle=True,
seed=42,
subset='validation'
)
test_images = test_generator.flow_from_dataframe(
dataframe=test_df,
x_col='Filepath',
y_col='Label',
target_size=(224, 224),
color_mode='rgb',
class_mode='categorical',
batch_size=32,
shuffle=False
)

Found 5656 validated image filenames belonging to 101 classes.
Found 1414 validated image filenames belonging to 101 classes.
Found 3030 validated image filenames belonging to 101 classes.


In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(  #loading a pretrained model that already knows how to recognize patterns in images. Let me break it down:
input_shape=(224, 224, 3),
include_top=False,
weights='imagenet',
pooling='avg'
)
pretrained_model.trainable = False

In [ ]:
# Improved model architecture with regularization
inputs = pretrained_model.input

# Add Global Average Pooling (already done by pooling='avg' in pretrained_model)
x = pretrained_model.output

# Add batch normalization and dropout for better regularization
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Dropout to prevent overfitting

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Output layer for 101 food categories
outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
print(model.summary())

In [ ]:
# Fine-tuning: Unfreeze the top layers of MobileNetV2 for better adaptation
# This allows the model to learn food-specific features
pretrained_model.trainable = True

# Freeze all layers except the last 30
for layer in pretrained_model.layers[:-30]:
    layer.trainable = False

print(f"Total layers in base model: {len(pretrained_model.layers)}")
print(f"Trainable layers: {sum([1 for layer in pretrained_model.layers if layer.trainable])}")

In [ ]:
# Compile with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Enhanced callbacks for better training
callbacks = [
    # Stop training when validation loss stops improving
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,  # Increased patience
        restore_best_weights=True,
        verbose=1
    ),
    # Reduce learning rate when validation loss plateaus
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Reduce LR by half
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    # Save the best model
    tf.keras.callbacks.ModelCheckpoint(
        'best_food_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train the model
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=callbacks
)

In [ ]:
# Visualize training history
plt.figure(figsize=(14, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Time')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"\nBest Validation Accuracy: {max(history.history['val_accuracy']):.2%}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.2%}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.2%}")

In [ ]:
# Evaluate the final model on test set
results = model.evaluate(test_images, verbose=1)
print("\n" + "="*50)
print("FINAL TEST RESULTS")
print("="*50)
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]*100:.2f}%")
print("="*50)

# Get predictions for detailed analysis
test_images.reset()
predictions = model.predict(test_images, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_images.classes
class_labels = list(test_images.class_indices.keys())

# Calculate top-5 accuracy
top5_acc = tf.keras.metrics.top_k_categorical_accuracy(
    test_images.labels, 
    predictions, 
    k=5
).numpy().mean()
print(f"\nTop-5 Accuracy: {top5_acc*100:.2f}%")
print("(Percentage of times the correct class is in the top 5 predictions)")

## Model Improvements Summary

### Changes Made to Increase Accuracy:

1. **Data Augmentation** ✓
   - Added rotation, shifting, shearing, zooming, and flipping
   - Helps model generalize better to new images

2. **Improved Architecture** ✓
   - Increased model capacity: 512 → 256 → 128 units
   - Added Batch Normalization for stable training
   - Added Dropout layers (0.5, 0.4, 0.3) to prevent overfitting

3. **Fine-Tuning** ✓
   - Unfroze top 30 layers of MobileNetV2
   - Allows model to learn food-specific features

4. **Better Training Strategy** ✓
   - Lower learning rate (0.0001) for fine-tuning
   - ReduceLROnPlateau to adapt learning rate
   - Increased patience for early stopping
   - Model checkpointing to save best weights

### Expected Improvements:
- **Previous accuracy**: ~42%
- **Expected accuracy**: 65-75% (with current improvements)
- **Top-5 accuracy**: Should be 85-90%

### Further Improvements (if needed):
- Try EfficientNetB0 or ResNet50 as base model
- Increase image size to 299x299
- Use more training epochs (75-100)
- Implement test-time augmentation
- Use ensemble of multiple models